**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-3vyz0826
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-3vyz0826
  Resolved https://github.com/Kaggle/learntools.git to commit 7a83948d1820d0282e518fc44692bb3118c9e1a6
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268963 sha256=8e4eeb4fa89b673044548aa0ddfd19c0d25dfbc4fc6ee3a2ad29b4202d9f3e7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-7f3tlqje/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,e6d4cffb601f697b44b6b0b64551821917b0a543,ffc2a618ad19a63fa5e9ce8b1bd68656172b2eac505f0a...,2013-01-17 19:00:00+00:00,2013-01-17 19:00:00+00:00,540,4.70,NaN,NaN,NaN,NaN,...,0.0,12.05,Cash,None,NaN,NaN,None,NaN,NaN,None
1,e85b82ecf9e825b474a903e2ca72e72a99987a57,e61ce97d61bec30e506e2ff56ea034ad7c06cc6a0a4c0f...,2013-01-25 23:15:00+00:00,2013-01-25 23:15:00+00:00,180,0.59,NaN,NaN,NaN,NaN,...,0.0,4.65,Cash,None,NaN,NaN,None,NaN,NaN,None
2,bd538c860ab1bbf471fd95d70381358903439270,e61ce97d61bec30e506e2ff56ea034ad7c06cc6a0a4c0f...,2013-01-25 19:30:00+00:00,2013-01-25 19:30:00+00:00,660,3.43,NaN,NaN,NaN,NaN,...,0.0,10.45,Cash,None,NaN,NaN,None,NaN,NaN,None
3,80af74ec9db84df289ff0d3e747fd0d704310686,e61ce97d61bec30e506e2ff56ea034ad7c06cc6a0a4c0f...,2013-01-25 19:45:00+00:00,2013-01-25 20:00:00+00:00,720,5.42,NaN,NaN,NaN,NaN,...,0.0,13.85,Cash,None,NaN,NaN,None,NaN,NaN,None
4,9b475199b49a6780ef1ca6053815962b9328223f,e61ce97d61bec30e506e2ff56ea034ad7c06cc6a0a4c0f...,2013-01-25 18:30:00+00:00,2013-01-25 18:30:00+00:00,480,1.33,NaN,NaN,NaN,NaN,...,1.0,7.45,Cash,None,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-02-15,91736.428571
1,2016-02-23,92244.285714
2,2016-02-05,81493.142857
3,2016-01-17,84366.285714
4,2016-02-27,92965.142857


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

```

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                        OVER(
                        PARTITION BY pickup_community_area
                        ORDER BY trip_start_timestamp
                        ) as trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,26.0,2013-10-03 04:30:00+00:00,2013-10-03 04:45:00+00:00,1
1,26.0,2013-10-03 05:15:00+00:00,2013-10-03 05:15:00+00:00,2
2,40.0,2013-10-03 09:00:00+00:00,2013-10-03 09:00:00+00:00,1
3,40.0,2013-10-03 20:00:00+00:00,2013-10-03 20:15:00+00:00,2
4,40.0,2013-10-03 20:00:00+00:00,2013-10-03 20:15:00+00:00,2


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

```

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [8]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,015b518d184aaf0d78dd3d6f5cf138fee49b3378558362...,2013-10-03 15:00:00+00:00,2013-10-03 15:45:00+00:00,-45.0
1,015b518d184aaf0d78dd3d6f5cf138fee49b3378558362...,2013-10-03 22:00:00+00:00,2013-10-03 22:00:00+00:00,270.0
2,081d71909728bc4f1c66aac975cae6f864bb57aafaffc7...,2013-10-03 06:30:00+00:00,2013-10-03 06:45:00+00:00,270.0
3,081d71909728bc4f1c66aac975cae6f864bb57aafaffc7...,2013-10-03 17:15:00+00:00,2013-10-03 17:30:00+00:00,360.0
4,13805c930c62981c661c5b3bcf8f6b1847b1a0a086a05b...,2013-10-03 07:45:00+00:00,2013-10-03 08:00:00+00:00,300.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [7]:
# Lines below will give you a hint or solution code
#q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

break_time_result = client.query(break_time_query).result().to_dataframe()

```

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*